import libraries and set up file handlers

In [ ]:
import simplejson, time
import codecs, StringIO
from proselint.command_line import lint

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

import scipy.stats
import pandas, seaborn

from string import punctuation
punctuation = list(punctuation)

sid = SentimentIntensityAnalyzer() #load the vadar analyzer

rh = codecs.open('data/RC_2015-01', encoding='utf8')
wh = codecs.open('data/per_comment_data_RC_2015-01.json', 'a', encoding='utf8')

Here's the core text processing

In [ ]:
    idx = 0
    errors = 0
    for raw_comment in rh:
        idx += 1
        if idx % 5000 == 0:
            print time.asctime(), "about to process comment", idx, 100*(idx/53851542.0), "percent done"
        
        comment = simplejson.loads(raw_comment)
        try:
            #comment token count
            num_tokens = len(filter(lambda t:t not in punctuation, tokenize.word_tokenize(comment['body'])))
            if num_tokens < 5:
                #sentiment/grammar not reliable for small comments, so skipping
                continue

            #lint the comment for style/grammar
            lints = lint(StringIO.StringIO(comment['body']))
            minimal_lints = map(lambda l:(l[0], l[7]), lints) #not storing explaination or position

            #vadar sentiment times
            ss = sid.polarity_scores(comment['body'])

           #write it out
            wh.write(simplejson.dumps({'id':comment['id'], 
                                       'lints':minimal_lints, 
                                       'sentiment':ss, 
                                       'num_tokens':num_tokens})+'\n')
            
        except Exception, err:
            print "error:", err, "while dealing with comment", comment['id'], "skipping"
            errors += 1

loading/processing the output file

In [ ]:
rh = codecs.open('data/per_comment_data_RC_2015-01.json',  encoding='utf8')
#screw it, just put the whole thing in memory
results = map(lambda r:simplejson.loads(r), rh) #text to objects
#build a pandas data frame
results_frame = pandas.DataFrame({'lints':map(lambda r:len(r['lints']), results), 
                                  'normed lints':map(lambda r:len(r['lints'])/float(r['num_tokens']), results),
                                  'compound':map(lambda r:r['sentiment']['compound'], results),
                                  'positive':map(lambda r:r['sentiment']['pos'], results),
                                  'neutral':map(lambda r:r['sentiment']['neu'], results),
                                  'negative':map(lambda r:r['sentiment']['neg'], results),
                                  'token counts':map(lambda r:r['num_tokens'], results),
                                  })
#basic stats from df describe
results_frame.describe()

#seaborn charting
seaborn.pairplot(results_frame, vars=["normed lints", "positive", "neutral", "negative"], kind='reg')

#pearsons correlation testing
scipy.stats.pearsonr(results_frame['negative'], results_frame['lints'])
scipy.stats.pearsonr(results_frame['negative'], results_frame['normed lints'])

    